In [ ]:
from dotenv import load_dotenv
import locale
import os
import pandas as pd
import sqlalchemy as sa

load_dotenv()

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")
locale.setlocale(locale.LC_MONETARY, "pt_BR.UTF-8")

pd.set_option("display.float_format", lambda x: locale.currency(x, symbol=False, grouping=True))

engine = sa.engine.create_engine(os.getenv("URL_MYSQL"))

In [ ]:
releases = [
    {"nm_curso": "..."},
]

df_releases = pd.DataFrame(releases)
rows_inserted = df_releases.to_sql(name="lançamentos", con=engine, if_exists="append", index=False)
print(f"Foram {rows_inserted} registros inseridos com sucesso.")

In [ ]:
df_releases = pd.read_sql(sql=sa.text("SELECT * FROM lançamento"), con=engine)
df_releases.columns = ["Código", "Lançamento"]
df_releases

In [ ]:
paychecks = [
    {"id_lançamento": 36, "período": 202406, "acerto": True, "valor": -84.31},
]

df_paychecks = pd.DataFrame(paychecks)
row_paychecks = df_paychecks.to_sql(name="espelho", con=engine, if_exists="append", index=False)
print(f"Foram {row_paychecks} registros inseridos com sucesso.")

In [ ]:
period = 202406

stmt = f"""
    SELECT
        l.lançamento,
        e.período,
        IF(e.acerto, 'acerto', 'mês') AS espelho,
        e.valor
    FROM
        espelho e
        INNER JOIN lançamento l ON e.id_lançamento = l.id_lançamento
    WHERE
        e.período = {period}
    ORDER BY
        e.acerto DESC
"""

df_mes = pd.read_sql(sql=sa.text(stmt), con=engine)
df_mes["período"] = pd.to_datetime(df_mes["período"], format="%Y%m").dt.strftime("%B de %Y")
df_mes

In [ ]:
ano = 2023

stmt = f"""
    SELECT
        l.lançamento,
        e.período,
        IF(e.acerto = 1, 'acerto', 'mês') AS acerto,
        e.valor
    FROM
        espelho e
        INNER JOIN lançamento l ON e.id_lançamento = l.id_lançamento
    WHERE
        e.período LIKE '{ano}%'
"""

df_ano = pd.read_sql(sql=sa.text(stmt), con=engine)

df_ano = df_ano.pivot(values=["valor"], index=["lançamento", "acerto"], columns=["período"])
df_ano.columns = df_ano.columns.droplevel(level=0)
df_ano.reset_index(inplace=True)
df_ano.fillna(value=0, inplace=True)
df_ano.sort_values(["acerto"], inplace=True, ignore_index=True)
df_ano.columns.rename("", inplace=True)
df_ano["média"] = df_ano.mean(axis=1, numeric_only=True)
df_ano["total"] = df_ano[df_ano.columns[1:]].sum(axis=1, numeric_only=True)
df_ano.loc["sumário"] = df_ano.sum(numeric_only=True)
df_ano.fillna(value="", inplace=True)
df_ano.iloc[-1, 0] = "Sumário"
df_ano.set_index(["lançamento"], inplace=True)
df_ano.rename(columns={int(f"{ano}01"): "jan", int(f"{ano}02"): "fev", int(f"{ano}03"): "mar", int(f"{ano}04"): "abr",
                       int(f"{ano}05"): "mai", int(f"{ano}06"): "jun", int(f"{ano}07"): "jul", int(f"{ano}08"): "ago",
                       int(f"{ano}09"): "set", int(f"{ano}10"): "out", int(f"{ano}11"): "nov", int(f"{ano}12"): "dez", }, inplace=True)
df_ano

In [ ]:
stmt = """
    SELECT
        SUBSTR(período, 1, 4) AS ano,
        CONCAT('mês ', SUBSTR(período, 5)) AS mes,
        SUM(valor) AS valor
    FROM
        espelho
    GROUP BY
        ano,
        mes
"""

df_anuais = pd.read_sql(sql=stmt, con=engine)

df_anuais = df_anuais.pivot(columns=["mes"], index=["ano"], values=["valor"])
df_anuais.columns = df_anuais.columns.droplevel(level=0)
df_anuais.reset_index(inplace=True)
df_anuais["ano"] = df_anuais["ano"].astype(int)
df_anuais.set_index(["ano"], inplace=True)
df_anuais.fillna(0, inplace=True)
df_anuais["média"] = df_anuais.mean(axis=1)
df_anuais["total"] = df_anuais[df_anuais.columns[:-1]].sum(axis=1)
df_anuais.columns = ["jan", "fev", "mar", "abr", "mai", "jun", "jul", "ago", "set", "out", "nov", "dez", "média", "total"]

df_anuais

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

ano = 2024

plt.figure(figsize=(16, 6))
plt.style.use("ggplot")

ax = sns.barplot(data=df_anuais.loc[[ano], df_anuais.columns[:-2]])
ax.set_title(f"Espelho {ano}", loc="center", fontweight="bold", fontsize=12)
ax.set(xlabel="", ylabel="", yticks=[])

for mes in range(12):
    ax.bar_label(ax.containers[mes], fmt=lambda i: locale.currency(i, symbol=False, grouping=True), fontsize=10)

plt.show()